In [4]:
import os
import glob
import numpy

data_files = []

def check_files(user_input):   
    """
    Checks to make sure given directory is a valid directory.  
    If directory exists, returns list of .fits files.
    """
    if os.path.exists(user_input) == True:
        os.chdir(directory)
        data_files = glob.glob("*.fit*")
        return data_files
    else:
        print("Invalid directory. Please try again.") 
        return False   

while True:
    directory = input("Enter Directory: ")   
    fits_files = check_files(directory)
    if fits_files != False:
        break

print(fits_files)

Enter Directory: /
[]


In [24]:
target_data = numpy.loadtxt(fname='/Users/jaredhand/Documents/Automation Project/Automation_Project/target_data.txt',
                            dtype=bytes,
                            delimiter=',')

def byte_to_str(list_input):
    for i in list_input:
        i.decode('utf-8')
    return list_input

target_name = target_data[:,0]
target_RA = target_data[:,1]
target_dec = target_data[:,2]

print(target_name)
print(byte_to_str(target_name))

def target_dict(name_list, val_list):
    """
    Takes list of names and list of values and returns a dictionary with with name_list as
    dictionary key.
    """
    result = dict(zip(name_list, val_list))
    return result

RA_dict = target_dict(target_name, target_RA)
print(RA_dict)
dec_dict = target_dict(target_name, target_dec)
print(dec_dict)

[b'PKS0048-097' b'PKS0113-118' b'DA55' b'3C66A' b'AO0235+164' b'BQCam'
 b'GKPer' b'LSV+44_17' b'V518Per' b'J06074+2205' b'PKS0528+134' b'V1055Ori'
 b'V616Mon' b'MXB0656-072' b'TXS0716+332' b'B30650+453' b'TXS0820+560'
 b'OJ425' b'OJ287' b'OJ49' b'PKS0907+022' b'PMN0948+002' b'PMN1016+051'
 b'PKS1127-14' b'Mrk421' b'KVUMa' b'B21215+30' b'WCom' b'B21308+32'
 b'3C273' b'3C279' b'PKS1329-049' b'PKS1352-104' b'J1357.2-0933'
 b'PKS1510-089' b'PKS1502+106' b'PKS1551+130' b'TXS1520+319' b'4C+38.41'
 b'Mrk501' b'OT355' b'TXS1846+322' b'V406Vul' b'OT081' b'J1753.5-0127'
 b'SerX-1' b'1846-031' b'J1855-026' b'4U1916-053' b'AqlX-1' b'J1901+014'
 b'J1856+053' b'SS433' b'J1908+094' b'4U1907+09' b'J19140+0951'
 b'4U1957+115' b'PKS2023-07' b'J1946+274' b'J1955+261' b'2000+251'
 b'V404Cyg' b'J2012+381' b'CygX-3' b'J2103+45' b'V490Cep' b'SSCyg' b'BLLac'
 b'CygX-2' b'PKS2144+092' b'OX169' b'PKS2201+171' b'CTA102' b'3C454.3'
 b'PKS2325+093' b'PHL5225' b'PMN2345-155']
[b'PKS0048-097' b'PKS0113-118' b'DA55' 

In [13]:
from astropy.table import Table, Column 

target_table = Table([target_name, target_RA, target_dec], 
                     names = ('name', 'RA', 'dec'))
print(target_table)

     name          RA         dec    
-------------- ---------- -----------
b'PKS0048-097'  b'0.8448'  b'-9.4848'
b'PKS0113-118'  b'1.2701' b'-11.6043'
       b'DA55'  b'1.6163'  b'47.8581'
      b'3C66A'  b'2.3777'  b'43.0355'
 b'AO0235+164'  b'2.6441'  b'16.6165'
      b'BQCam'  b'3.5833'  b'53.1732'
      b'GKPer'  b'3.5200'  b'43.9043'
  b'LSV+44_17'  b'4.6831'  b'44.5304'
    b'V518Per'  b'4.3620'  b'32.9082'
b'J06074+2205'  b'6.1241'  b'22.0966'
           ...        ...         ...
      b'SSCyg' b'21.7119'  b'43.5861'
      b'BLLac' b'22.0454'  b'42.2778'
     b'CygX-2' b'21.7448'  b'38.3214'
b'PKS2144+092' b'21.7861'   b'9.4963'
      b'OX169' b'21.7266'  b'17.7301'
b'PKS2201+171' b'22.0575'  b'17.4301'
     b'CTA102' b'22.5434'  b'11.7308'
    b'3C454.3' b'22.8994'  b'16.1482'
b'PKS2325+093' b'23.4593'   b'9.6694'
    b'PHL5225' b'22.4945'  b'-8.5485'
b'PMN2345-155' b'23.7534' b'-15.9188'
Length = 77 rows


In [6]:
for file in fits_files:
    break

0
